# TCP Client Implementation

> Building a TCP client with detailed explanations of each step

In [ ]:
#| default_exp client

## Introduction

In this notebook, we'll implement a TCP client that:
1. Creates a socket
2. Connects to a server
3. Sends and receives data
4. Closes the connection gracefully

Let's import the necessary modules:

In [ ]:
#| export
from python_tcp.core import *
import socket
from typing import Optional, List, Tuple, Dict, Any, Union, Callable
import threading
import time

## Simple TCP Client

We'll start with a basic TCP client implementation:

In [ ]:
#| export
class TCPClient:
    """A simple TCP client for connecting to TCP servers."""
    
    def __init__(self, buffer_size: int = DEFAULT_BUFFER_SIZE):
        """Initialize the client."""
        self.buffer_size = buffer_size
        self.sock = None
        self.state = SocketState.CLOSED
        self.connected = False
        self.connection = None
        self.receive_thread = None
    
    def connect(self, host: str, port: int) -> bool:
        """Connect to a TCP server at the specified host and port."""
        if self.connected:
            print("Already connected to a server")
            return False
        
        try:
            # Create a TCP socket
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            
            # Set socket options
            self.sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            
            # Update state to SYN_SENT (simulating TCP handshake)
            self.state = SocketState.SYN_SENT
            print(f"Connecting to {host}:{port}...")
            
            # Connect to the server
            self.sock.connect((host, port))
            
            # Connected successfully, update state
            self.state = SocketState.ESTABLISHED
            self.connected = True
            
            # Create a connection object to track this connection
            self.connection = TCPConnection(
                sock=self.sock,
                state=self.state,
                remote_address=(host, port),
                connection_id="client-connection"
            )
            
            print(f"Connected to {host}:{port}")
            return True
        except Exception as e:
            print(f"Error connecting to {host}:{port}: {e}")
            self.close()
            return False
    
    def send(self, data: bytes) -> bool:
        """Send data to the connected server."""
        if not self.connected or not self.sock:
            print("Not connected to a server")
            return False
        
        try:
            self.sock.sendall(data)
            return True
        except Exception as e:
            print(f"Error sending data: {e}")
            self.close()
            return False
    
    def receive(self) -> Optional[bytes]:
        """Receive data from the server (blocking call)."""
        if not self.connected or not self.sock:
            print("Not connected to a server")
            return None
        
        try:
            data = self.sock.recv(self.buffer_size)
            if not data:
                # Empty data means the server closed the connection
                print("Server closed the connection")
                self.close()
                return None
            
            return data
        except Exception as e:
            print(f"Error receiving data: {e}")
            self.close()
            return None
    
    def close(self) -> None:
        """Close the connection to the server."""
        if self.sock:
            try:
                # Update state to simulate TCP termination
                if self.state == SocketState.ESTABLISHED:
                    self.state = SocketState.FIN_WAIT_1
                
                self.sock.close()
                print("Connection closed")
            except Exception as e:
                print(f"Error closing connection: {e}")
        
        self.sock = None
        self.connected = False
        self.state = SocketState.CLOSED
        
        if self.connection:
            self.connection.update_state(SocketState.CLOSED)
            self.connection = None

## Enhanced TCP Client with Asynchronous Receive

Now let's create a more advanced client with asynchronous message reception:

In [ ]:
#| export
class AsyncTCPClient(TCPClient):
    """A TCP client with asynchronous message reception in a background thread."""
    
    def __init__(self, buffer_size: int = DEFAULT_BUFFER_SIZE):
        """Initialize the asynchronous client."""
        super().__init__(buffer_size)
        self.receive_callback: Optional[Callable[[bytes], None]] = None
        self.error_callback: Optional[Callable[[Exception], None]] = None
        self.running = False
    
    def set_receive_callback(self, callback: Callable[[bytes], None]) -> None:
        """Set a callback function to handle received data."""
        self.receive_callback = callback
    
    def set_error_callback(self, callback: Callable[[Exception], None]) -> None:
        """Set a callback function to handle errors."""
        self.error_callback = callback
    
    def connect(self, host: str, port: int) -> bool:
        """Connect to a server and start the receive thread."""
        if not super().connect(host, port):
            return False
        
        # Start the receive thread
        self.running = True
        self.receive_thread = threading.Thread(target=self._receive_loop)
        self.receive_thread.daemon = True
        self.receive_thread.start()
        
        return True
    
    def _receive_loop(self) -> None:
        """Continuously receive data in a background thread."""
        while self.running and self.connected:
            try:
                data = self.sock.recv(self.buffer_size)
                
                if not data:
                    # Empty data means the server closed the connection
                    print("Server closed the connection")
                    break
                
                # Call the receive callback if set
                if self.receive_callback:
                    try:
                        self.receive_callback(data)
                    except Exception as e:
                        print(f"Error in receive callback: {e}")
                        if self.error_callback:
                            self.error_callback(e)
            except Exception as e:
                print(f"Error receiving data: {e}")
                if self.error_callback:
                    self.error_callback(e)
                break
            
        # When the loop exits, close the connection
        self.close()
    
    def close(self) -> None:
        """Close the connection and stop the receive thread."""
        self.running = False
        
        # Wait for the receive thread to finish
        if self.receive_thread and self.receive_thread.is_alive():
            self.receive_thread.join(timeout=1.0)
        
        super().close()

## Event-Driven TCP Client

Finally, let's create an event-driven TCP client with detailed connection state events:

In [ ]:
#| export
class EventDrivenTCPClient(AsyncTCPClient):
    """A TCP client that emits events for connection state changes."""
    
    def __init__(self, buffer_size: int = DEFAULT_BUFFER_SIZE):
        """Initialize the event-driven client."""
        super().__init__(buffer_size)
        # Event callbacks
        self.on_connect: Optional[Callable[[str, int], None]] = None
        self.on_disconnect: Optional[Callable[[], None]] = None
        self.on_data: Optional[Callable[[bytes], None]] = None
        self.on_error: Optional[Callable[[Exception], None]] = None
        
        # Auto-set callbacks based on events
        self.set_receive_callback(lambda data: self._on_data_received(data))
        self.set_error_callback(lambda error: self._on_error(error))
    
    def connect(self, host: str, port: int) -> bool:
        """Connect to a server and trigger the on_connect event."""
        if super().connect(host, port):
            # Trigger on_connect event
            if self.on_connect:
                try:
                    self.on_connect(host, port)
                except Exception as e:
                    print(f"Error in on_connect callback: {e}")
            return True
        return False
    
    def close(self) -> None:
        """Close the connection and trigger the on_disconnect event."""
        was_connected = self.connected
        
        super().close()
        
        # Trigger on_disconnect event
        if was_connected and self.on_disconnect:
            try:
                self.on_disconnect()
            except Exception as e:
                print(f"Error in on_disconnect callback: {e}")
    
    def _on_data_received(self, data: bytes) -> None:
        """Internal handler for received data that triggers the on_data event."""
        if self.on_data:
            try:
                self.on_data(data)
            except Exception as e:
                print(f"Error in on_data callback: {e}")
    
    def _on_error(self, error: Exception) -> None:
        """Internal handler for errors that triggers the on_error event."""
        if self.on_error:
            try:
                self.on_error(error)
            except Exception as e:
                print(f"Error in on_error callback: {e}")

In [ ]:
c = EventDrivenTCPClient()
c.connect(LOCALHOST, 8000)
c.send(b"Hello, server!")
c.receive()
c.close()


Connecting to 127.0.0.1:8000...
Error connecting to 127.0.0.1:8000: [Errno 111] Connection refused
Connection closed
Not connected to a server
Not connected to a server


## Example Usage

Here's a simple example of how to use our basic TCP client:

In [ ]:
def simple_client_demo():
    # Create a TCP client
    client = TCPClient()
    
    # Connect to a server
    if client.connect(LOCALHOST, 8000):
        try:
            # Send a message
            client.send(b"Hello, server!")
            
            # Receive the response
            response = client.receive()
            if response:
                print(f"Received from server: {response.decode('utf-8')}")
            
            # Send another message
            client.send(b"How are you?")
            
            # Receive the response
            response = client.receive()
            if response:
                print(f"Received from server: {response.decode('utf-8')}")
                
        finally:
            # Close the connection
            client.close()

# Uncomment to run the demo (make sure a server is running)
# simple_client_demo()

Let's see a more advanced example with the event-driven client:

In [ ]:
def event_driven_client_demo():
    # Create an event-driven TCP client
    client = EventDrivenTCPClient()
    
    # Set up event handlers
    client.on_connect = lambda host, port: print(f"EVENT: Connected to {host}:{port}")
    client.on_disconnect = lambda: print("EVENT: Disconnected from server")
    client.on_data = lambda data: print(f"EVENT: Received data: {data.decode('utf-8')}")
    client.on_error = lambda error: print(f"EVENT: Error occurred: {error}")
    
    # Connect to a server
    if client.connect(LOCALHOST, 8000):
        try:
            # Send some messages
            client.send(b"Hello from event-driven client!")
            time.sleep(1)
            
            client.send(b"This client uses events for processing responses.")
            time.sleep(1)
            
            # Wait for a while to receive responses
            print("Waiting for responses...")
            time.sleep(3)
            
        finally:
            # Close the connection
            client.close()

# Uncomment to run the demo (make sure a server is running)
# event_driven_client_demo()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()